In [31]:
from mtcnn import MTCNN
from PIL import Image
from os import listdir
from os.path import isdir
from numpy import asarray

import pandas as pd
from numpy import expand_dims
from sklearn.utils import shuffle
from numpy import asarray

from tensorflow.keras.models import load_model

detector = MTCNN()
ok = "ok\n"

In [32]:
class Getting_embedds:

    def flip_image(image):

        img = image.transpose(Image.FLIP_LEFT_RIGHT)

        return img

In [33]:
def extrair_face(arquivo, size = (160,160)):
    
    img = Image.open(arquivo)
    img = img.convert('RGB')

    array = asarray(img)

    results = detector.detect_faces(array)

    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height

    face = array[y1:y2, x1:x2]

    image = Image.fromarray(face)
    image = image.resize(size)

    return image

In [34]:
def load_fotos():
    allfaces = list()
    directory_scr = r"C:\Users\VCHAGAS\Documents\Python Scripts\Face-Recognition-main\fotos\\"
    
    for filename in listdir(directory_scr):
        
        path = directory_scr + filename

        try:
            face = extrair_face(path)
            face_flip = flip_image(face)

            face = face.convert("RGB")
            face_flip = face_flip.convert("RGB")

            allfaces.append(asarray(face))
            allfaces.append(asarray(face_flip))

        except:
            print(f"\nerror na imagem {path}")
            
    allfaces = asarray(allfaces)
    
    return allfaces

In [35]:
def get_embedding(model, face_pixels):
    
    #Padronização
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean)/std
    
    #transformar a face em 1 unico exemplo (160,160) -> (1,160,160)
    
    samples = expand_dims(face_pixels, axis=0)
    
    #Realizar a predição gerando o embedding
    yhat = model.predict(samples)
    
    return yhat[0]

In [6]:
if __name__ == "__main__":
    
    trainX = load_fotos()
    newTrainX = list()
    morador = input("Digite o nome do morador: ")
    
    model = load_model("facenet_keras.h5")
    
    for face_pixels in trainX:
        embedding = get_embedding(model, face_pixels)
        newTrainX.append(embedding)
    
    newTrainX = asarray(newTrainX)
    df = pd.DataFrame(data = newTrainX)
    
    trainy = [morador]*40
    df['target'] = trainy
    
    df.to_csv('novo_morador.csv')
    
    df.head()
    
    

1/1 [==============================] - ETA: 0s

C:\Users\VCHAGAS\Anaconda3\envs\help\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  This is separate from the ipykernel package so we can avoid doing imports until


1/1 [==============================] - 0s 15ms/step
Digite o nome do morador: vitor
1/1 [==============================] - 0s 45ms/step
